# Aula05.Ex02- Exemplo - Esfera - Transformação Geométrica 3D

### Primeiro, vamos importar as bibliotecas necessárias.
Verifique no código anterior um script para instalar as dependências necessárias (OpenGL e GLFW) antes de prosseguir.

In [1]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np

### Inicializando janela

In [2]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
window = glfw.create_window(700, 700, "Esfera", None, None)
glfw.make_context_current(window)

### GLSL (OpenGL Shading Language)

Aqui veremos nosso primeiro código GLSL.

É uma linguagem de shading de alto nível baseada na linguagem de programação C.

Nós estamos escrevendo código GLSL como se "strings" de uma variável (mas podemos ler de arquivos texto). Esse código, depois, terá que ser compilado e linkado ao nosso programa. 

Iremos aprender GLSL conforme a necessidade do curso. Usarmos uma versão do GLSL mais antiga, compatível com muitos dispositivos.

### GLSL para Vertex Shader

No Pipeline programável, podemos interagir com Vertex Shaders.

No código abaixo, estamos fazendo o seguinte:

* Definindo uma variável chamada position do tipo vec2.
* Definindo uma variável chamada mat_transformation do tipo mat4 (matriz 4x4). Use ela como matriz de transformação, resultante de uma sequências de outras transformações (e.g. rotação + translação)
* Usamos vec2, pois nosso programa (na CPU) irá enviar apenas duas coordenadas para plotar um ponto. Podemos mandar três coordenadas (vec3) e até mesmo quatro coordenadas (vec4).
* void main() é o ponto de entrada do nosso programa (função principal)
* gl_Position é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a posição de um vértice. Observe que todo vértice tem 4 coordenadas, por isso nós combinamos nossa variável vec2 com uma variável vec4. Além disso, nós modificamos nosso vetor com base em uma matriz de transformação.

In [3]:
vertex_code = """
        attribute vec3 position;
        uniform mat4 mat_transformation;
        void main(){
            gl_Position = mat_transformation * vec4(position,1.0);
        }
        """

### GLSL para Fragment Shader

No Pipeline programável, podemos interagir com Fragment Shaders.

No código abaixo, estamos fazendo o seguinte:

* void main() é o ponto de entrada do nosso programa (função principal)
* gl_FragColor é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a cor de um fragmento. Nesse caso é um ponto, mas poderia ser outro objeto (ponto, linha, triangulos, etc).

### Possibilitando modificar a cor.

Nos exemplos anteriores, a variável gl_FragColor estava definida de forma fixa (com cor R=0, G=0, B=0).

Agora, nós vamos criar uma variável do tipo "uniform", de quatro posições (vec4), para receber o dado de cor do nosso programa rodando em CPU.

In [4]:
fragment_code = """
        uniform vec4 color;
        void main(){
            gl_FragColor = color;
        }
        """

### Requisitando slot para a GPU para nossos programas Vertex e Fragment Shaders

In [5]:
# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)


### Associando nosso código-fonte aos slots solicitados

In [6]:
# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

### Compilando o Vertex Shader

Se há algum erro em nosso programa Vertex Shader, nosso app para por aqui.

In [7]:
# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")


### Compilando o Fragment Shader

Se há algum erro em nosso programa Fragment Shader, nosso app para por aqui.

In [8]:
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

### Associando os programas compilado ao programa principal

In [9]:
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)


### Linkagem do programa

In [10]:
# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)

### Preparando dados para enviar a GPU

Nesse momento, nós compilamos nossos Vertex e Program Shaders para que a GPU possa processá-los.

Por outro lado, as informações de vértices geralmente estão na CPU e devem ser transmitidas para a GPU.


### Modelando uma Esfera



In [11]:
import math


PI = 3.141592
r = 0.5 # raio
num_sectors = 32 # qtd de sectors (longitude)
num_stacks = 32 # qtd de stacks (latitude)


# grid sectos vs stacks (longitude vs latitude)
sector_step=(PI*2)/num_sectors # variar de 0 até 2π
stack_step=(PI)/num_stacks # variar de 0 até π

# Entrada: angulo de longitude, latitude, raio
# Saida: coordenadas na esfera
def F(u,v,r):
    x = r*math.sin(v)*math.cos(u)
    y = r*math.sin(v)*math.sin(u)
    z = r*math.cos(v)
    return (x,y,z)

# vamos gerar um conjunto de vertices representantes poligonos
# para a superficie da esfera.
# cada poligono eh representado por dois triangulos
vertices_list = []
for i in range(0,num_sectors): # para cada sector (longitude)
    for j in range(0,num_stacks): # para cada stack (latitude)
        
        
        
        u = i * sector_step # angulo setor
        v = j * stack_step # angulo stack
        
        un = 0 # angulo do proximo sector
        if i+1==num_sectors:
            un = PI*2
        else: un = (i+1)*sector_step
            
        vn = 0 # angulo do proximo stack
        if j+1==num_stacks:
            vn = PI
        else: vn = (j+1)*stack_step
        
        # vertices do poligono
        p0=F(u, v, r)
        p1=F(u, vn, r)
        p2=F(un, v, r)
        p3=F(un, vn, r)
        
        # triangulo 1 (primeira parte do poligono)
        vertices_list.append(p0)
        vertices_list.append(p2)
        vertices_list.append(p1)
        
        # triangulo 2 (segunda e ultima parte do poligono)
        vertices_list.append(p3)
        vertices_list.append(p1)
        vertices_list.append(p2)


total_vertices = len(vertices_list)
vertices = np.zeros(total_vertices, [("position", np.float32, 3)])
vertices['position'] = np.array(vertices_list)

In [12]:
for v in vertices:
    print(v)

([0.0, 0.0, 0.5],)
([0.0, 0.0, 0.5],)
([0.04900856, 0.0, 0.49759236],)
([0.048066873, 0.0095610935, 0.49759236],)
([0.04900856, 0.0, 0.49759236],)
([0.0, 0.0, 0.5],)
([0.04900856, 0.0, 0.49759236],)
([0.048066873, 0.0095610935, 0.49759236],)
([0.09754514, 0.0, 0.49039266],)
([0.09567084, 0.019030109, 0.49039266],)
([0.09754514, 0.0, 0.49039266],)
([0.048066873, 0.0095610935, 0.49759236],)
([0.09754514, 0.0, 0.49039266],)
([0.09567084, 0.019030109, 0.49039266],)
([0.1451423, 0.0, 0.47847018],)
([0.14235345, 0.028315853, 0.47847018],)
([0.1451423, 0.0, 0.47847018],)
([0.09567084, 0.019030109, 0.49039266],)
([0.1451423, 0.0, 0.47847018],)
([0.14235345, 0.028315853, 0.47847018],)
([0.19134168, 0.0, 0.46193978],)
([0.1876651, 0.037328903, 0.46193978],)
([0.19134168, 0.0, 0.46193978],)
([0.14235345, 0.028315853, 0.47847018],)
([0.19134168, 0.0, 0.46193978],)
([0.1876651, 0.037328903, 0.46193978],)
([0.23569833, 0.0, 0.44096065],)
([0.23116945, 0.045982454, 0.44096065],)
([0.23569833, 0.0, 0.

([-0.19042017,  0.4597155 , -0.0490084 ],)
([-0.09567069,  0.48096997, -0.09754498],)
([-0.18766497,  0.45306385, -0.09754498],)
([-0.09567069,  0.48096997, -0.09754498],)
([-0.19042017,  0.4597155 , -0.0490084 ],)
([-0.09567069,  0.48096997, -0.09754498],)
([-0.18766497,  0.45306385, -0.09754498],)
([-0.09334474,  0.46927658, -0.14514215],)
([-0.18310244,  0.4420489 , -0.14514215],)
([-0.09334474,  0.46927658, -0.14514215],)
([-0.18766497,  0.45306385, -0.09754498],)
([-0.09334474,  0.46927658, -0.14514215],)
([-0.18310244,  0.4420489 , -0.14514215],)
([-0.09011982,  0.45306385, -0.19134153],)
([-0.17677656,  0.42677683, -0.19134153],)
([-0.09011982,  0.45306385, -0.19134153],)
([-0.18310244,  0.4420489 , -0.14514215],)
([-0.09011982,  0.45306385, -0.19134153],)
([-0.17677656,  0.42677683, -0.19134153],)
([-0.08602701,  0.43248782, -0.23569818],)
([-0.1687482 ,  0.40739468, -0.23569818],)
([-0.08602701,  0.43248782, -0.23569818],)
([-0.17677656,  0.42677683, -0.19134153],)
([-0.086027

([-0.3840889 ,  0.1590951 , -0.27778494],)
([-0.43248776,  0.08602744, -0.23569818],)
([-0.3840889 ,  0.1590951 , -0.27778494],)
([-0.40774664,  0.08110611, -0.27778494],)
([-0.35708433,  0.1479094 , -0.31719646],)
([-0.37907875,  0.07540369, -0.31719646],)
([-0.35708433,  0.1479094 , -0.31719646],)
([-0.40774664,  0.08110611, -0.27778494],)
([-0.35708433,  0.1479094 , -0.31719646],)
([-0.37907875,  0.07540369, -0.31719646],)
([-0.3266408 ,  0.13529928, -0.3535532 ],)
([-0.3467601 ,  0.06897509, -0.3535532 ],)
([-0.3266408 ,  0.13529928, -0.3535532 ],)
([-0.37907875,  0.07540369, -0.31719646],)
([-0.3266408 ,  0.13529928, -0.3535532 ],)
([-0.3467601 ,  0.06897509, -0.3535532 ],)
([-0.2930516 ,  0.12138614, -0.38650507],)
([-0.31110194,  0.06188222, -0.38650507],)
([-0.2930516 ,  0.12138614, -0.38650507],)
([-0.3467601 ,  0.06897509, -0.3535532 ],)
([-0.2930516 ,  0.12138614, -0.38650507],)
([-0.31110194,  0.06188222, -0.38650507],)
([-0.25664014,  0.10630399, -0.41573465],)
([-0.272447

([-0.12068174, -0.08063683, -0.4784701 ],)
([-0.10263141, -0.10263125, -0.4784701 ],)
([-0.12068174, -0.08063683, -0.4784701 ],)
([-0.13529932, -0.1352991 , -0.46193966],)
([-0.12068174, -0.08063683, -0.4784701 ],)
([-0.10263141, -0.10263125, -0.4784701 ],)
([-0.08110613, -0.05419329, -0.49039257],)
([-0.06897511, -0.068975  , -0.49039257],)
([-0.08110613, -0.05419329, -0.49039257],)
([-0.10263141, -0.10263125, -0.4784701 ],)
([-0.08110613, -0.05419329, -0.49039257],)
([-0.06897511, -0.068975  , -0.49039257],)
([-0.04074942, -0.02722785, -0.49759233],)
([-0.03465454, -0.03465449, -0.49759233],)
([-0.04074942, -0.02722785, -0.49759233],)
([-0.06897511, -0.068975  , -0.49039257],)
([-0.04074942, -0.02722785, -0.49759233],)
([-0.03465454, -0.03465449, -0.49759233],)
([-2.7172015e-07, -1.8155731e-07, -5.0000000e-01],)
([-2.3107907e-07, -2.3107870e-07, -5.0000000e-01],)
([-2.7172015e-07, -1.8155731e-07, -5.0000000e-01],)
([-0.03465454, -0.03465449, -0.49759233],)
([-0. , -0. ,  0.5],)
([-0.

([ 0.13529865, -0.32664084,  0.35355344],)
([ 0.07540303, -0.37907866,  0.31719673],)
([ 0.14790875, -0.35708436,  0.31719673],)
([ 0.08110541, -0.4077466 ,  0.2777852 ],)
([ 0.1590944, -0.384089 ,  0.2777852],)
([ 0.08110541, -0.4077466 ,  0.2777852 ],)
([ 0.14790875, -0.35708436,  0.31719673],)
([ 0.08110541, -0.4077466 ,  0.2777852 ],)
([ 0.1590944, -0.384089 ,  0.2777852],)
([ 0.0860267 , -0.43248773,  0.23569846],)
([ 0.16874787, -0.40739465,  0.23569846],)
([ 0.0860267 , -0.43248773,  0.23569846],)
([ 0.1590944, -0.384089 ,  0.2777852],)
([ 0.0860267 , -0.43248773,  0.23569846],)
([ 0.16874787, -0.40739465,  0.23569846],)
([ 0.0901195 , -0.45306376,  0.19134183],)
([ 0.17677623, -0.42677683,  0.19134183],)
([ 0.0901195 , -0.45306376,  0.19134183],)
([ 0.16874787, -0.40739465,  0.23569846],)
([ 0.0901195 , -0.45306376,  0.19134183],)
([ 0.17677623, -0.42677683,  0.19134183],)
([ 0.09334441, -0.46927655,  0.14514247],)
([ 0.18310212, -0.44204897,  0.14514247],)
([ 0.09334441, -0.46

([ 0.46927643, -0.09334551, -0.14514215],)
([ 0.42677656, -0.17677724, -0.19134153],)
([ 0.4530637 , -0.09012057, -0.19134153],)
([ 0.42677656, -0.17677724, -0.19134153],)
([ 0.46927643, -0.09334551, -0.14514215],)
([ 0.42677656, -0.17677724, -0.19134153],)
([ 0.4530637 , -0.09012057, -0.19134153],)
([ 0.40739438, -0.16874887, -0.23569818],)
([ 0.4324877 , -0.08602772, -0.23569818],)
([ 0.40739438, -0.16874887, -0.23569818],)
([ 0.4530637 , -0.09012057, -0.19134153],)
([ 0.40739438, -0.16874887, -0.23569818],)
([ 0.4324877 , -0.08602772, -0.23569818],)
([ 0.38408878, -0.15909535, -0.27778494],)
([ 0.4077466 , -0.08110638, -0.27778494],)
([ 0.38408878, -0.15909535, -0.27778494],)
([ 0.4324877 , -0.08602772, -0.23569818],)
([ 0.38408878, -0.15909535, -0.27778494],)
([ 0.4077466 , -0.08110638, -0.27778494],)
([ 0.3570842 , -0.14790964, -0.31719646],)
([ 0.3790787 , -0.07540394, -0.31719646],)
([ 0.3570842 , -0.14790964, -0.31719646],)
([ 0.4077466 , -0.08110638, -0.27778494],)
([ 0.357084

In [13]:
len(vertices)

6144

### Para enviar nossos dados da CPU para a GPU, precisamos requisitar um slot.

In [14]:
# Request a buffer slot from GPU
buffer = glGenBuffers(1)
# Make this buffer the default one
glBindBuffer(GL_ARRAY_BUFFER, buffer)


### Abaixo, nós enviamos todo o conteúdo da variável vertices.

Veja os parâmetros da função glBufferData [https://www.khronos.org/registry/OpenGL-Refpages/gl4/html/glBufferData.xhtml]

In [15]:
# Upload data
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
glBindBuffer(GL_ARRAY_BUFFER, buffer)

### Associando variáveis do programa GLSL (Vertex Shaders) com nossos dados

Primeiro, definimos o byte inicial e o offset dos dados.

In [16]:
# Bind the position attribute
# --------------------------------------
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)


Em seguida, soliciamos à GPU a localização da variável "position" (que guarda coordenadas dos nossos vértices). Nós definimos essa variável no Vertex Shader.

In [17]:
loc = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc)

A partir da localização anterior, nós indicamos à GPU onde está o conteúdo (via posições stride/offset) para a variável position (aqui identificada na posição loc).

Outros parâmetros:

* Definimos que possui duas coordenadas
* Que cada coordenada é do tipo float (GL_FLOAT)
* Que não se deve normalizar a coordenada (False)

Mais detalhes: https://www.khronos.org/registry/OpenGL-Refpages/gl4/html/glVertexAttribPointer.xhtml

In [18]:
glVertexAttribPointer(loc, 3, GL_FLOAT, False, stride, offset)

###  Vamos pegar a localização da variável color (uniform) do Fragment Shader para que possamos alterá-la em nosso laço da janela!

In [19]:
loc_color = glGetUniformLocation(program, "color")

### Nesse momento, nós exibimos a janela!


In [20]:
glfw.show_window(window)

### Loop principal da janela.
Enquanto a janela não for fechada, esse laço será executado. É neste espaço que trabalhamos com algumas interações com a OpenGL.


Usaremos o GL_TRIANGLE para modelar os triângulos (que formarão outros polígonos) da superfície da nossa esfera.

In [21]:
import math
import random
d = 0.0
glEnable(GL_DEPTH_TEST) ### importante para 3D

def multiplica_matriz(a,b):
    m_a = a.reshape(4,4)
    m_b = b.reshape(4,4)
    m_c = np.dot(m_a,m_b)
    c = m_c.reshape(1,16)
    return c

while not glfw.window_should_close(window):

    glfw.poll_events() 
    
    
    ### apenas para visualizarmos a esfera rotacionando
    d -= 0.001 # modifica o angulo de rotacao em cada iteracao
    
    cos_d = math.cos(d)
    sin_d = math.sin(d)
    
    
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    
    glClearColor(1.0, 1.0, 1.0, 1.0)
    
    mat_rotation_z = np.array([     cos_d, -sin_d, 0.0, 0.0, 
                                    sin_d,  cos_d, 0.0, 0.0, 
                                    0.0,      0.0, 1.0, 0.0, 
                                    0.0,      0.0, 0.0, 1.0], np.float32)
    
    mat_rotation_x = np.array([     1.0,   0.0,    0.0, 0.0, 
                                    0.0, cos_d, -sin_d, 0.0, 
                                    0.0, sin_d,  cos_d, 0.0, 
                                    0.0,   0.0,    0.0, 1.0], np.float32)
    
    mat_rotation_y = np.array([     cos_d,  0.0, sin_d, 0.0, 
                                    0.0,    1.0,   0.0, 0.0, 
                                    -sin_d, 0.0, cos_d, 0.0, 
                                    0.0,    0.0,   0.0, 1.0], np.float32)
    
    

    
    mat_transform = multiplica_matriz(mat_rotation_z,mat_rotation_x)
    mat_transform = multiplica_matriz(mat_rotation_y,mat_transform)

    loc = glGetUniformLocation(program, "mat_transformation")
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transform)

    #glPolygonMode(GL_FRONT_AND_BACK,GL_LINE)
    
    for triangle in range(0,len(vertices),3):
       
        random.seed( triangle )
        R = random.random()
        G = random.random()
        B = random.random()  
#         R = 0
#         G = 0
#         B = 0
        glUniform4f(loc_color, R, G, B, 1.0)
        glDrawArrays(GL_TRIANGLES, triangle, 3)

    glfw.swap_buffers(window)

glfw.terminate()

# Exercício

Modifique esse código para aplicar outras transformações (de sua escolha) conforme eventos do teclado.